In [ ]:
# https://gseapy.readthedocs.io/en/latest/gseapy_example.html

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp

In [2]:
enrichr_adj_pval_thresh = 0.05

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
dbs = gp.get_library_name()


ConnectionError: HTTPConnectionPool(host='maayanlab.cloud', port=80): Max retries exceeded with url: /Enrichr/datasetStatistics (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x147876d10>: Failed to resolve 'maayanlab.cloud' ([Errno 8] nodename nor servname provided, or not known)"))

In [5]:
go_bp = gp.get_library(name = 'GO_Biological_Process_2023', organism = 'human')

ConnectionError: HTTPConnectionPool(host='maayanlab.cloud', port=80): Max retries exceeded with url: /Enrichr/datasetStatistics (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x148cdf010>: Failed to resolve 'maayanlab.cloud' ([Errno 8] nodename nor servname provided, or not known)"))

## Read in naming file

In [ ]:
#inputFile = 'data/omics_revamped_LLM_DF.tsv'
inputFile = 'omics_revamped_LLM_Enrichr_simVals_failure_refs_newThresh_DF.tsv'

In [ ]:
#outputFile = 'data/omics_revamped_LLM_Enrichr_DF.tsv' 
outputFile = 'data/omics_revamped_LLM_Enrichr_simVals_failure_refs_newThresh___test_reviwers.tsv' 

In [ ]:
genesets_LLM = pd.read_csv(inputFile, sep = "\t") 

In [ ]:
# genesets_LLM.GeneSetID.unique()

In [ ]:
# Remove "HALLMARK_" prefix for MSigDB gene sets
genesets_LLM['GeneSetName'] = genesets_LLM['GeneSetName'].str.removeprefix("HALLMARK_").str.replace('_', ' ')


In [ ]:
columnsToKeep = ['Source', 'GeneSetID', 'GeneSetName', 'GeneList', 'n_Genes', 'LLM Name',
       'LLM Analysis', 'Score']
geneSeparator = " "

In [ ]:
LLM_DF = genesets_LLM

In [ ]:
nGeneSets = LLM_DF.shape[0]

In [ ]:
LLM_repeated_DF = LLM_DF[LLM_DF[['Source', 'GeneSetID', 'GeneSetName', 'GeneList']].duplicated() == True]


## Go through each system and run Enrichr 

In [ ]:
# Each system will have multiple rows

In [ ]:
additionalCols = ['Rank', 'Overlap', 'P-value', 'Adjusted P-value', 'Genes', 'Term', 'GO term', 'GO ID' ]

In [ ]:
expandedColumnsToKeep =columnsToKeep;
expandedColumnsToKeep.extend(additionalCols)

In [ ]:
expanded_LLM_genes_DF = pd.DataFrame(columns=expandedColumnsToKeep)

In [ ]:
genesCol = 'GeneList'

In [ ]:
for geneSetInd in range(nGeneSets):
    print(geneSetInd)
    geneSetRow = LLM_DF.iloc[geneSetInd]
    geneSetGenes = geneSetRow[genesCol].split(geneSeparator)

    # Perform enrichment analysis
    enr = gp.enrichr(gene_list=geneSetGenes, # or "./tests/data/gene_list.txt",
                 gene_sets=['GO_Biological_Process_2023'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )
    
    # threshold
    # sigRes_DF =  enr.results.loc[enr.results['Adjusted P-value'] <=0.1]
    sigRes_DF = enr.results # remove thresholding, because want a name for each one
    sigRes_DF = sigRes_DF.sort_values('Adjusted P-value')
    if sigRes_DF.shape[0] == 0:
        
        print([geneSetInd, ' has not enirchment'])
        #continue # skip to next gene set
        sigRes_DF.loc[0, ['Term', 'GO term', 'GO ID', 'nothing']] = 'NaN'
        sigRes_DF.loc[0, ['Rank']]  = 1
        sigRes_DF.loc[0, ['Overlap']]  = 0
        sigRes_DF.loc[0, ['P-value']]  = 1
        sigRes_DF.loc[0, ['Adjusted P-value']]  = 1
        sigRes_DF.loc[0, ['Genes']]  = ''
        
    else:
        expandedCols = sigRes_DF["Term"].str.split('(', expand = True)
        
        if expandedCols.shape[1] == 3:
            sigRes_DF[['GO term', 'GO ID', 'nothing']] = expandedCols
        else: # ==2
            sigRes_DF[['GO term', 'GO ID']] = expandedCols

        sigRes_DF[['GO ID', 'nothing']] = sigRes_DF["GO ID"].str.split(')', expand = True)
        

        # Get ranking of enriched GO terms
        sigRes_DF['Rank'] = sigRes_DF.reset_index().index

        if sigRes_DF.loc[0, 'Adjusted P-value'] > enrichr_adj_pval_thresh: # nothing significant
            sigRes_DF = sigRes_DF.loc[[0],:]
        else: # there are sig. terms
            sigRes_DF = sigRes_DF[sigRes_DF['Adjusted P-value'] <=enrichr_adj_pval_thresh]
        
            
        
    
    # Doing what should be a simple colbind 
    geneSetRow_DF = geneSetRow.to_frame().T
    geneSetRow_repeated_DF = geneSetRow_DF.loc[geneSetRow_DF.index.repeat(sigRes_DF.shape[0])]
    geneSetRow_repeated_DF = geneSetRow_repeated_DF.reset_index()
    
    # system_Row_sigRes = systemRow_repeated_DF.join(sigRes_DF)
    geneSetRowRow_sigRes = pd.concat([geneSetRow_repeated_DF, sigRes_DF],axis=1)
    
    # Doing what should be a simple  rowbind
    if expanded_LLM_genes_DF.shape[0] == 0:
        expanded_LLM_genes_DF = geneSetRowRow_sigRes[expandedColumnsToKeep]
    else:
        expanded_LLM_genes_DF =  pd.concat([expanded_LLM_genes_DF, geneSetRowRow_sigRes[expandedColumnsToKeep]],axis=0)
    
    if (geneSetInd%10 == 1):
        expanded_LLM_genes_DF.to_csv(outputFile, sep = "\t") # Save to not loose work

In [ ]:
def get_goterm_genes(goterm):
    if goterm in go_bp:
        return " ".join(go_bp[goterm])
    else:
        return ""

In [ ]:
expanded_LLM_genes_DF['GO_term_genes'] = [get_goterm_genes(goterm) for goterm in expanded_LLM_genes_DF['Term']]

In [ ]:
expanded_LLM_genes_DF.to_csv(outputFile, sep = "\t") # Too large 

In [ ]:
expanded_LLM_genes_copy_DF = expanded_LLM_genes_DF;

In [ ]:
def select_min_rows(df, group_vars, min_var):
    # Convert the target column to numeric, non-numeric values become NaN
    df[min_var] = pd.to_numeric(df[min_var], errors='coerce')

    # Reset index of the DataFrame
    df_reset = df.reset_index()

    # Get the indices of the minimum values in the grouped DataFrame
    min_indices = df_reset.groupby(group_vars)[min_var].idxmin()

    # Use these indices to select rows from the reset DataFrame
    return df_reset.loc[min_indices].set_index('index')

In [ ]:
reduced_LLM_genes_DF = select_min_rows(expanded_LLM_genes_copy_DF, ['Source', 'GeneSetID', 'GeneSetName', 'GeneList'], 'Adjusted P-value')

In [ ]:
reduced_LLM_genes_DF.to_csv('data/omics_revamped_LLM_Enrichr_reduced_DF.tsv' , sep = "\t")

In [ ]:
set(LLM_DF.GeneSetID.to_list()).difference(set(reduced_LLM_genes_DF.GeneSetID.to_list()))

In [ ]:
# reduced_LLM_genes_DF[reduced_LLM_genes_DF[['Source', 'GeneSetID', 'GeneSetName', 'GeneList']].duplicated() == True]
